In [32]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)
# engine = create_engine('sqlite:///database.sqlite3', convert_unicode=True, echo=True)

from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
metadata = MetaData()
corpus = Table('corpus', metadata,
              Column('id', Integer, primary_key=True),
              Column('index', Integer),
              Column('chapter', String),
              Column('text', String),
              Column('translate', String),
              Column('translite', String),
              Column('audio', String),
              Column('lang', String),
)

In [33]:
metadata.drop_all(engine)

2020-01-06 15:29:33,720 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-01-06 15:29:33,723 INFO sqlalchemy.engine.base.Engine ()
2020-01-06 15:29:33,729 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-01-06 15:29:33,732 INFO sqlalchemy.engine.base.Engine ()
2020-01-06 15:29:33,738 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("corpus")
2020-01-06 15:29:33,743 INFO sqlalchemy.engine.base.Engine ()
2020-01-06 15:29:33,749 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("corpus")
2020-01-06 15:29:33,754 INFO sqlalchemy.engine.base.Engine ()


In [34]:
metadata.create_all(engine)

2020-01-06 15:29:35,995 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("corpus")
2020-01-06 15:29:35,998 INFO sqlalchemy.engine.base.Engine ()
2020-01-06 15:29:36,001 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("corpus")
2020-01-06 15:29:36,006 INFO sqlalchemy.engine.base.Engine ()
2020-01-06 15:29:36,011 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE corpus (
	id INTEGER NOT NULL, 
	"index" INTEGER, 
	chapter VARCHAR, 
	text VARCHAR, 
	translate VARCHAR, 
	translite VARCHAR, 
	audio VARCHAR, 
	lang VARCHAR, 
	PRIMARY KEY (id)
)


2020-01-06 15:29:36,014 INFO sqlalchemy.engine.base.Engine ()
2020-01-06 15:29:36,016 INFO sqlalchemy.engine.base.Engine COMMIT


In [35]:
import glob
corpus_prefix='/pi/stack/crawlers/langcrs'
ds=[f for f in glob.glob(f'{corpus_prefix}/all_*.json')]
ds

['/pi/stack/crawlers/langcrs/all_no.json',
 '/pi/stack/crawlers/langcrs/all_lv.json',
 '/pi/stack/crawlers/langcrs/all_da.json',
 '/pi/stack/crawlers/langcrs/all_af.json',
 '/pi/stack/crawlers/langcrs/all_id.json',
 '/pi/stack/crawlers/langcrs/all_cs.json',
 '/pi/stack/crawlers/langcrs/all_fa.json',
 '/pi/stack/crawlers/langcrs/all_vi.json',
 '/pi/stack/crawlers/langcrs/all_es.json',
 '/pi/stack/crawlers/langcrs/all_ar.json',
 '/pi/stack/crawlers/langcrs/all_sv.json',
 '/pi/stack/crawlers/langcrs/all_fr.json',
 '/pi/stack/crawlers/langcrs/all_et.json',
 '/pi/stack/crawlers/langcrs/all_he.json',
 '/pi/stack/crawlers/langcrs/all_ko.json',
 '/pi/stack/crawlers/langcrs/all_ro.json',
 '/pi/stack/crawlers/langcrs/all_bg.json',
 '/pi/stack/crawlers/langcrs/all_ca.json',
 '/pi/stack/crawlers/langcrs/all_hi.json',
 '/pi/stack/crawlers/langcrs/all_el.json',
 '/pi/stack/crawlers/langcrs/all_ur.json',
 '/pi/stack/crawlers/langcrs/all_fi.json',
 '/pi/stack/crawlers/langcrs/all_de.json',
 '/pi/stack

In [36]:
def load_corpus(langs):
    rs=[]
    for lang in langs:
        with open(f'/pi/stack/crawlers/langcrs/all_{lang}.json') as json_file:
            sents=json.load(json_file)
            for s in sents:
                s['lang']=lang
            rs.extend(sents)
    return rs
        
conn = engine.connect()
rs=load_corpus(['fr', 'es', 'ja'])
print(rs[:3])
conn.execute(corpus.insert(), rs)

[{'chapter': 'Reading and writing', 'index': 1, 'text': 'I read.', 'translate': 'Je lis.', 'translit': '', 'audio': 'https://www.book2.nl/book2/FR/SOUND/0103.mp3', 'lang': 'fr'}, {'chapter': 'Reading and writing', 'index': 2, 'text': 'I read a letter (character).', 'translate': 'Je lis une lettre.', 'translit': '', 'audio': 'https://www.book2.nl/book2/FR/SOUND/0104.mp3', 'lang': 'fr'}, {'chapter': 'Reading and writing', 'index': 3, 'text': 'I read a word.', 'translate': 'Je lis un mot.', 'translit': '', 'audio': 'https://www.book2.nl/book2/FR/SOUND/0105.mp3', 'lang': 'fr'}]
2020-01-06 15:29:43,013 INFO sqlalchemy.engine.base.Engine INSERT INTO corpus ("index", chapter, text, translate, audio, lang) VALUES (?, ?, ?, ?, ?, ?)
2020-01-06 15:29:43,014 INFO sqlalchemy.engine.base.Engine ((1, 'Reading and writing', 'I read.', 'Je lis.', 'https://www.book2.nl/book2/FR/SOUND/0103.mp3', 'fr'), (2, 'Reading and writing', 'I read a letter (character).', 'Je lis une lettre.', 'https://www.book2.nl

In [38]:
from sqlalchemy.sql import select
from sqlalchemy.sql import and_, or_, not_

# s = select([corpus])
# result = conn.execute(s)
chapter='People'
# lang='es'
lang='ja'
s = select([corpus]).where(and_ (corpus.c.chapter==chapter,
                               corpus.c.lang==lang))
for row in conn.execute(s):
    print(row)

2020-01-06 15:30:10,615 INFO sqlalchemy.engine.base.Engine SELECT corpus.id, corpus."index", corpus.chapter, corpus.text, corpus.translate, corpus.translite, corpus.audio, corpus.lang 
FROM corpus 
WHERE corpus.chapter = ? AND corpus.lang = ?
2020-01-06 15:30:10,617 INFO sqlalchemy.engine.base.Engine ('People', 'ja')
(3601, 1, 'People', 'I', '私', None, 'https://www.book2.nl/book2/JA/SOUND/0003.mp3', 'ja')
(3602, 2, 'People', 'I and you', '私と あなた', None, 'https://www.book2.nl/book2/JA/SOUND/0004.mp3', 'ja')
(3603, 3, 'People', 'both of us', '私達', None, 'https://www.book2.nl/book2/JA/SOUND/0005.mp3', 'ja')
(3604, 4, 'People', 'he', '彼', None, 'https://www.book2.nl/book2/JA/SOUND/0006.mp3', 'ja')
(3605, 5, 'People', 'he and she', '彼と 彼女', None, 'https://www.book2.nl/book2/JA/SOUND/0007.mp3', 'ja')
(3606, 6, 'People', 'they both', '彼ら', None, 'https://www.book2.nl/book2/JA/SOUND/0008.mp3', 'ja')
(3607, 7, 'People', 'the man', '男性', None, 'https://www.book2.nl/book2/JA/SOUND/0009.mp3', 'ja'